In [1]:
import dill
import numpy
import librosa
import torch
import sklearn
import scipy
import fastdtw


In [1]:
######## to dill ########

import os
import pdb
import scipy.io.wavfile
import dill
import sys
#sys.path.append('E:\\Codes_For_Python\\.vscode\\Huang')
from create_data.pitch_histogram import CreateNoteHistogram
from matplotlib import pylab as plt
import numpy as np

# mother_dir = "/data07/chitra/workspace/data/PESnQ+Discover_DATA"
mother_dir = "D:\DAMP\Research"
SingEval_DAMPcurated_dir = "D:\DAMP\SingEval_DAMPcurated"
songs = ['_cups_pitch_perfect','_let_it_go','_stay_rihanna','_when_i_was_your_man']
audio_snippets = ['audio_snippets_1','audio_snippets_2','audio_snippets_3','audio_snippets_4','audio_snippets_5']

test_singers = [1, 11, 21, 31, 41, 51, 61, 71, 81, 91]
val_singers = [2, 12, 22, 32, 42, 52, 62, 72, 82, 92]
train_singers = list(set(range(100)) - set(test_singers) - set(val_singers))


def CreateBWSdict(bwsfile):
    ## This function takes in bws file and outputs a dictionary with the singername and the corresponding score
    flines = open(bwsfile, 'r').readlines()
    names_scores = []
    for line in flines:
        name, score = line.replace('\n', '').split(',')
        names_scores.append((name, float(score)))
    names_scores.sort(key=lambda x: (-x[1], x[0]))
    SingerScore = {}
    for name, score in names_scores:
        SingerScore[name] = score
    return names_scores, SingerScore


def plotpitch(pitch):
    plt.figure()
    plt.plot(pitch)
    plt.show()

# cv1_folder = '/data07/huanglin/SingEval/LeaderboardData/data_ph'
#cv1_folder = 'D:/nuspro/dilldata_ph/test'
cv1_folder = mother_dir +  os.sep +'dilldata'
train_dill = open(cv1_folder + os.sep + 'train_1.dill', 'wb')
test_dill = open(cv1_folder + os.sep + 'test_1.dill', 'wb')
val_dill = open(cv1_folder + os.sep + 'val_1.dill', 'wb')

train_tobedumped = []
test_tobedumped = []
val_tobedumped = []

for songname in songs:
    print(songname)
    bwsfile = mother_dir+os.sep+'bws' +os.sep+'bws' + songname + '.txt'
    ### Create dictionary of Human BWS Scores
    singer_score_tuple_sorted, singer_score_dict = CreateBWSdict(bwsfile)
    # pitch_folder = '/data07/huanglin/SingEval/LeaderboardData/pitch_folder'
    pitch_folder =  mother_dir +  os.sep +'pitch_folder'

    for idx in train_singers:
        singername = singer_score_tuple_sorted[idx][0]
        rating = singer_score_tuple_sorted[idx][1]
        for snippet_num in audio_snippets:
            wavfile = SingEval_DAMPcurated_dir + os.sep + snippet_num + os.sep + songname + os.sep + singername + '.wav'
            fs, audio = scipy.io.wavfile.read(wavfile)
            original_pitch_file = pitch_folder + os.sep + snippet_num + '_' + singername + '.pitch'
            ph_notes = CreateNoteHistogram(original_pitch_file)# 已经生成了histogram然后放入dill中
            train_tobedumped.append({'audio': [audio / 32768.0, fs], 'pitch_histogram': ph_notes, 'ratings': [rating]})


    for idx in test_singers:
        singername = singer_score_tuple_sorted[idx][0]
        rating = singer_score_tuple_sorted[idx][1]
        for snippet_num in audio_snippets:
            wavfile = SingEval_DAMPcurated_dir + os.sep + snippet_num + os.sep + songname + os.sep + singername + '.wav'
            fs, audio = scipy.io.wavfile.read(wavfile)
            original_pitch_file = pitch_folder + os.sep + snippet_num + '_' + singername + '.pitch'
            ph_notes = CreateNoteHistogram(original_pitch_file)
            test_tobedumped.append({'audio': [audio / 32768.0, fs], 'pitch_histogram': ph_notes, 'ratings': [rating]})




    for idx in val_singers:
        singername = singer_score_tuple_sorted[idx][0]
        rating = singer_score_tuple_sorted[idx][1]
        for snippet_num in audio_snippets:
            wavfile = SingEval_DAMPcurated_dir + os.sep + snippet_num + os.sep + songname + os.sep + singername + '.wav'
            fs, audio = scipy.io.wavfile.read(wavfile)
            original_pitch_file = pitch_folder + os.sep + snippet_num + '_' + singername + '.pitch'
            ph_notes = CreateNoteHistogram(original_pitch_file)
            val_tobedumped.append({'audio': [audio / 32768.0, fs], 'pitch_histogram': ph_notes, 'ratings': [rating]})

dill.dump(train_tobedumped, train_dill)
dill.dump(test_tobedumped, test_dill)
dill.dump(val_tobedumped, val_dill)
train_dill.close()
test_dill.close()
val_dill.close()

_cups_pitch_perfect
_let_it_go
_stay_rihanna
_when_i_was_your_man


In [ ]:
#train  Cqt ph
import torch
import time
import numpy as np
from torch import nn
from torch.autograd import Variable
from models.SpectralCRNN import SpectralCRNN_Reg_Dropout_tsne as SpectralCRNN
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from sklearn import metrics
from torch.optim import lr_scheduler
from scipy.stats import pearsonr as pearsoncorr
from scipy.stats import spearmanr as spearmancorr
import pdb
import random
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from tensorboard_logger import configure, log_value
# Configure tensorboard logger

# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
configure('runs/Mel_Spectrogram_CRNN', flush_secs=2)  
# Parameteres for Spectral Representation
rep_params = {'method': 'Mel Spectrogram', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}

# Load Datasets
train_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_train_1.dill', 0, rep_params)
train_dataloader = SpectralDataLoader(train_dataset, batch_size=5, num_workers=4, shuffle=True)
#pdb.set_trace()

test_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_test_1.dill', 0, rep_params)
test_dataloader = SpectralDataLoader(test_dataset, batch_size=1, num_workers=1,shuffle=True)

valid_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_val_1.dill', 0, rep_params)
valid_dataloader = SpectralDataLoader(valid_dataset, batch_size=1, num_workers=1,shuffle=True)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def evaluate_classification(targets, predictions):
    r2 = metrics.r2_score(targets, predictions)
    predictions[predictions < -1] = -1
    predictions[predictions > 1] = 1
    pearson = pearsoncorr(targets.flatten(), predictions.flatten())
    pearson_corre = pearson[0]
    spearman = spearmancorr(targets.flatten(), predictions.flatten())
    print(f'Pearson Corr =  {pearson_corre:9.5f},  P-value = {pearson[1]}')
    print(f"Spearman Corr =  {spearman[0]:7.5f},  P-value =  {spearman[1]}")
    print("R2 = ", r2)
    return r2, pearson_corre,spearman[0]


# Define Model
model = SpectralCRNN().cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)

batch_time = AverageMeter()
data_time = AverageMeter()

train_loss = 0
validation_loss = 0

num_epochs = 250
best_val = 0.0
epoch_time = time.time()
for epoch in range(num_epochs):
    print('\n')
    print('Epoch: ', epoch)
    print('training set: ')
    model.train()
    avg_loss = 0.0
    end = time.time()
    all_predictions = []
    all_targets = []
    losses = AverageMeter()
    for i, (data) in enumerate(train_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end)
        end = time.time()
    train_loss = losses.avg
    train_r2, train_pearson, train_spearman= evaluate_classification(np.array(all_targets), np.array(all_predictions))

    print('validation set: ')
    model.eval()
    losses = AverageMeter()
    all_predictions = []
    all_targets = []
    for i, (data) in enumerate(valid_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
    #print('Validating Completed. Loss: {}'.format(losses.avg))
    valid_loss = losses.avg
    val_r2, val_pearson,val_spearman  = evaluate_classification(np.array(all_targets), np.array(all_predictions))
    print('Train Loss', train_loss, epoch)
    print('Validation Loss', valid_loss, epoch)
    log_value('Train Loss', train_loss, epoch)
    log_value('Validation Loss', valid_loss, epoch)
    log_value('Training R2', train_r2, epoch)
    log_value('Validation R2', val_r2, epoch)
    log_value('Training Spearman Corr ', train_spearman, epoch)
    log_value('Validation Spearman Corr ', val_spearman, epoch)
    log_value('Training Pearson Corr ', train_pearson, epoch)
    log_value('Validation Pearson Corr ', val_pearson, epoch)
    if val_pearson > best_val:
        best_val = val_pearson
        print('save model: ')
        torch.save(model, './models_hybrid/Mel_Spectrogram_CRNN')

In [1]:
##      train CQT_ph  ##

import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import time
import numpy as np
from torch import nn
from torch.autograd import Variable
from models.SpectralCRNN_hybrid import SpectralCRNN_Reg_Dropout_CQT_ph as SpectralCRNN
from tensorboard_logger import configure, log_value
from dataLoaders.SpectralDataset_ph import SpectralDataset, SpectralDataLoader
from sklearn import metrics
from torch.optim import lr_scheduler
from scipy.stats import pearsonr as pearsoncorr
from scipy.stats import spearmanr as spearmancorr
import pdb
import random



# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def evaluate_classification(targets, predictions):
    r2 = metrics.r2_score(targets, predictions)
    predictions[predictions < -1] = -1
    predictions[predictions > 1] = 1
    pearson = pearsoncorr(targets.flatten(), predictions.flatten())
    pearson_corre = pearson[0]
    spearman = spearmancorr(targets.flatten(), predictions.flatten())
    print(f'Pearson Corr =  {pearson_corre:9.5f},  P-value = {pearson[1]}')
    print(f"Spearman Corr =  {spearman[0]:7.5f},  P-value =  {spearman[1]}")
    print(f"R2 =  {r2:7.5f}") #r2->1
    return r2, pearson_corre,spearman[0]

def main():
    # Configure tensorboard logger
    configure('runs/CQT_CRNN_', flush_secs=2)  
    ###Anoconda Prompt:  tensorboard --logdir D:\DAMP\Research\runs\CQT_CRNN_pt --port 12

    #Parameteres for Spectral Representation
    rep_params = {'method': 'CQT', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}

    # Load Datasets
    train_dataset = SpectralDataset('D:/DAMP/Research/dilldata/train_1.dill', 0, rep_params)
    train_dataloader = SpectralDataLoader(train_dataset, batch_size=5, num_workers=4, shuffle=True)
    # pdb.set_trace()

    test_dataset = SpectralDataset('D:/DAMP/Research/dilldata/test_1.dill', 0, rep_params)
    test_dataloader = SpectralDataLoader(test_dataset, batch_size=1, num_workers=1, shuffle=True)

    valid_dataset = SpectralDataset('D:/DAMP/Research/dilldata/val_1.dill', 0, rep_params)
    valid_dataloader = SpectralDataLoader(valid_dataset, batch_size=1, num_workers=1, shuffle=True)

    # Define Model
    model = SpectralCRNN().cuda()

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)

    batch_time = AverageMeter()
    data_time = AverageMeter()

    train_loss = 0
    validation_loss = 0

    num_epochs = 250 #250
    best_val_r2 = 0.0
    best_val_corr = 0.0
    epoch_time = time.time()
    for epoch in range(num_epochs):
        print('\n')
        print('Epoch:', epoch)
        print('training set: ')
        model.train()
        # scheduler.step()
        avg_loss = 0.0
        end = time.time()
        all_predictions_all = []
        all_predictions_ps = []
        all_targets = []
        losses = AverageMeter()
        for i, (data) in enumerate(train_dataloader):
            inputs, targets, ph_notes = data
            #inputs, targets, ph_notes,pitchscore = data
            data_time.update(time.time() - end)
            inputs = Variable(inputs.cuda(), requires_grad=False)
            targets = Variable(targets.cuda(), requires_grad=False)
            targets = targets.view(-1, 1)
           # print(targets)
            # pitchscore = Variable(pitchscore.cuda(), requires_grad=False)
            # pitchscore = pitchscore.view(-1, 1)
            
            PH = Variable(ph_notes.cuda(), requires_grad=False)
            model.init_hidden(inputs.size(0))
            result,_ = model(inputs, PH)
           # out_all = result.view(-1,1)
            out_all = result[:,:1].view(-1,1)
            #print(out_all)
            all_predictions_all.extend(out_all.data.cpu().numpy())
            all_targets.extend(targets.data.cpu().numpy())

            
            loss = criterion(out_all, targets) 
            loss_value = loss.data
            losses.update(loss_value, inputs.size(0))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch_time.update(time.time() - end)
            end = time.time()
        train_loss = losses.avg
        train_r2, train_pearson, train_spearman = evaluate_classification(np.array(all_targets), np.array(all_predictions_all))

        print('validation set: ')
        model.eval()
        losses = AverageMeter()
        all_predictions = []
        all_targets = []
        for i, (data) in enumerate(valid_dataloader):
            inputs, targets, ph_notes = data
            data_time.update(time.time() - end)
            inputs = Variable(inputs.cuda(), requires_grad=False)
            targets = Variable(targets.cuda(), requires_grad=False)
            targets = targets.view(-1, 1)
           # print(targets)
            PH = Variable(ph_notes.cuda(), requires_grad=False)
            model.init_hidden(inputs.size(0))
            result,_= model(inputs, PH)
            #out = result.view(-1,1)
            out = result
            
           # out = model(inputs, PH)[0][0].view(-1,1)
            all_predictions.extend(out.data.cpu().numpy())
            all_targets.extend(targets.data.cpu().numpy())
            
            loss = criterion(out, targets)
            loss_value = loss.data
            losses.update(loss_value, inputs.size(0))
        valid_loss = losses.avg
        val_r2, val_pearson,val_spearman = evaluate_classification(np.array(all_targets), np.array(all_predictions))
        print('Train Loss', train_loss, epoch)
        print('Validation Loss', valid_loss, epoch)
        log_value('Train Loss', train_loss, epoch)
        log_value('Validation Loss', valid_loss, epoch)
        log_value('Training R2', train_r2, epoch)
        log_value('Validation R2', val_r2, epoch)
        log_value('Training Spearman Corr ', train_spearman, epoch)
        log_value('Validation Spearman Corr ', val_spearman, epoch)
        log_value('Training Pearson Corr ', train_pearson, epoch)
        log_value('Validation Pearson Corr ', val_pearson, epoch)
    # if val_r2 > best_val_r2:
        # print('Training R2', train_r2, epoch)
        # print('Validation R2', val_r2, epoch)
        # if val_r2 > best_val_r2:
        #     best_val_r2 = val_r2
        #     print('save r2 model')
        #     torch.save(model, './models_hybrid/model_SpectralCRNN_reg_lr0.0001_big_ELU_Adam_noteacc_bws-1to1_CQT_ph_r2')

        if val_pearson > best_val_corr:
            best_val_corr = val_pearson
            print('save pearson corr model: ')
            torch.save(model, './models_hybrid/CQT_CRNN_try')
    


if __name__=='__main__':
    main()



c:\Users\User\anaconda3\envs\ailab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Epoch: 0
training set: 
Pearson Corr =    0.20768,  P-value = 4.757747345011346e-17
Spearman Corr =  0.20770,  P-value =  4.733464285196039e-17
R2 =  -0.01677
validation set: 
Pearson Corr =    0.48115,  P-value = 5.521433545648314e-13
Spearman Corr =  0.47560,  P-value =  1.1042802927337937e-12
R2 =  0.08624
Train Loss tensor(0.2016, device='cuda:0') 0
Validation Loss tensor(0.1745, device='cuda:0') 0


ValueError: default logger is not configured. Call tensorboard_logger.configure(logdir), or use tensorboard_logger.Logger

In [1]:
#train  Cqt ph
import torch
import time
import numpy as np
from torch import nn
from torch.autograd import Variable
from models.SpectralCRNN import SpectralCRNN_Reg_Dropout_tsne as SpectralCRNN
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from sklearn import metrics
from torch.optim import lr_scheduler
from scipy.stats import pearsonr as pearsoncorr
from scipy.stats import spearmanr as spearmancorr
import pdb
import random
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from tensorboard_logger import configure, log_value
# Configure tensorboard logger

# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
configure('runs/CQT_CRNN', flush_secs=2)  
# Parameteres for Spectral Representation
rep_params = {'method': 'CQT', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}

# Load Datasets
train_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_train_1.dill', 0, rep_params)
train_dataloader = SpectralDataLoader(train_dataset, batch_size=5, num_workers=4, shuffle=True)
#pdb.set_trace()

test_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_test_1.dill', 0, rep_params)
test_dataloader = SpectralDataLoader(test_dataset, batch_size=1, num_workers=1,shuffle=True)

valid_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_val_1.dill', 0, rep_params)
valid_dataloader = SpectralDataLoader(valid_dataset, batch_size=1, num_workers=1,shuffle=True)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def evaluate_classification(targets, predictions):
    r2 = metrics.r2_score(targets, predictions)
    predictions[predictions < -1] = -1
    predictions[predictions > 1] = 1
    pearson = pearsoncorr(targets.flatten(), predictions.flatten())
    pearson_corre = pearson[0]
    spearman = spearmancorr(targets.flatten(), predictions.flatten())
    print(f'Pearson Corr =  {pearson_corre:9.5f},  P-value = {pearson[1]}')
    print(f"Spearman Corr =  {spearman[0]:7.5f},  P-value =  {spearman[1]}")
    print("R2 = ", r2)
    return r2, pearson_corre,spearman[0]


# Define Model
model = SpectralCRNN().cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)

batch_time = AverageMeter()
data_time = AverageMeter()

train_loss = 0
validation_loss = 0

num_epochs = 250
best_val = 0.0
epoch_time = time.time()
for epoch in range(num_epochs):
    print('\n')
    print('Epoch: ', epoch)
    print('training set: ')
    model.train()
    avg_loss = 0.0
    end = time.time()
    all_predictions = []
    all_targets = []
    losses = AverageMeter()
    for i, (data) in enumerate(train_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end)
        end = time.time()
    train_loss = losses.avg
    train_r2, train_pearson, train_spearman= evaluate_classification(np.array(all_targets), np.array(all_predictions))

    print('validation set: ')
    model.eval()
    losses = AverageMeter()
    all_predictions = []
    all_targets = []
    for i, (data) in enumerate(valid_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
    #print('Validating Completed. Loss: {}'.format(losses.avg))
    valid_loss = losses.avg
    val_r2, val_pearson,val_spearman  = evaluate_classification(np.array(all_targets), np.array(all_predictions))
    print('Train Loss', train_loss, epoch)
    print('Validation Loss', valid_loss, epoch)
    log_value('Train Loss', train_loss, epoch)
    log_value('Validation Loss', valid_loss, epoch)
    log_value('Training R2', train_r2, epoch)
    log_value('Validation R2', val_r2, epoch)
    log_value('Training Spearman Corr ', train_spearman, epoch)
    log_value('Validation Spearman Corr ', val_spearman, epoch)
    log_value('Training Pearson Corr ', train_pearson, epoch)
    log_value('Validation Pearson Corr ', val_pearson, epoch)
    if val_pearson > best_val:
        best_val = val_pearson
        print('save model: ')
        torch.save(model, './models_hybrid/CQT_CRNN')



Epoch:  0
training set: 
Pearson Corr =    0.17511,  P-value = 1.7452895839094653e-12
Spearman Corr =  0.18866,  P-value =  2.7657724628597024e-14
R2 =  -0.07813044370036759
validation set: 
Pearson Corr =    0.44486,  P-value = 4.11918006498314e-11
Spearman Corr =  0.43726,  P-value =  9.548058523736894e-11
R2 =  0.1845923676618555
Train Loss tensor(0.2138, device='cuda:0') 0
Validation Loss tensor(0.1557, device='cuda:0') 0
save model: 


Epoch:  1
training set: 
Pearson Corr =    0.31032,  P-value = 4.678735871390918e-37
Spearman Corr =  0.30196,  P-value =  4.3825448495230366e-35
R2 =  0.07217322800778392
validation set: 
Pearson Corr =    0.51594,  P-value = 5.334138390418772e-15
Spearman Corr =  0.52368,  P-value =  1.7646863647945382e-15
R2 =  0.03543839821411254
Train Loss tensor(0.1840, device='cuda:0') 1
Validation Loss tensor(0.1842, device='cuda:0') 1
save model: 


Epoch:  2
training set: 
Pearson Corr =    0.38138,  P-value = 1.49839867103306e-56
Spearman Corr =  0.3803

KeyboardInterrupt: 

In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.synchronize()
print(torch.cuda.memory_allocated())

0


In [1]:
#train  Cqt0611
import torch
import time
import numpy as np
from torch import nn
from torch.autograd import Variable
from models.SpectralCRNN import SpectralCRNN_Reg_Dropout_tsne as SpectralCRNN
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from sklearn import metrics
from torch.optim import lr_scheduler
from scipy.stats import pearsonr as pearsoncorr
from scipy.stats import spearmanr as spearmancorr
import pdb
import random
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from tensorboard_logger import configure, log_value
# Configure tensorboard logger

# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
configure('runs/CQT_CRNN_3', flush_secs=2)  
# Parameteres for Spectral Representation
rep_params = {'method': 'CQT', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}

# Load Datasets
train_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_train_1.dill', 0, rep_params)
train_dataloader = SpectralDataLoader(train_dataset, batch_size=5, num_workers=4, shuffle=True)
#pdb.set_trace()

test_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_test_1.dill', 0, rep_params)
test_dataloader = SpectralDataLoader(test_dataset, batch_size=1, num_workers=1,shuffle=True)

valid_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_val_1.dill', 0, rep_params)
valid_dataloader = SpectralDataLoader(valid_dataset, batch_size=1, num_workers=1,shuffle=True)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def evaluate_classification(targets, predictions):
    r2 = metrics.r2_score(targets, predictions)
    predictions[predictions < -1] = -1
    predictions[predictions > 1] = 1
    pearson = pearsoncorr(targets.flatten(), predictions.flatten())
    pearson_corre = pearson[0]
    spearman = spearmancorr(targets.flatten(), predictions.flatten())
    print(f'Pearson Corr =  {pearson_corre:9.5f},  P-value = {pearson[1]}')
    print(f"Spearman Corr =  {spearman[0]:7.5f},  P-value =  {spearman[1]}")
    print("R2 = ", r2)
    return r2, pearson_corre,spearman[0]


# Define Model
model = SpectralCRNN().cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)

batch_time = AverageMeter()
data_time = AverageMeter()

train_loss = 0
validation_loss = 0

num_epochs = 150
best_val = 0.0
epoch_time = time.time()
for epoch in range(num_epochs):
    print('\n')
    print('Epoch: ', epoch)
    print('training set: ')
    model.train()
    avg_loss = 0.0
    end = time.time()
    all_predictions = []
    all_targets = []
    losses = AverageMeter()
    for i, (data) in enumerate(train_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end)
        end = time.time()
    train_loss = losses.avg
    train_r2, train_pearson, train_spearman= evaluate_classification(np.array(all_targets), np.array(all_predictions))

    print('validation set: ')
    model.eval()
    losses = AverageMeter()
    all_predictions = []
    all_targets = []
    for i, (data) in enumerate(valid_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
    #print('Validating Completed. Loss: {}'.format(losses.avg))
    valid_loss = losses.avg
    val_r2, val_pearson,val_spearman  = evaluate_classification(np.array(all_targets), np.array(all_predictions))
    print('Train Loss', train_loss, epoch)
    print('Validation Loss', valid_loss, epoch)
    log_value('Train Loss', train_loss, epoch)
    log_value('Validation Loss', valid_loss, epoch)
    log_value('Training R2', train_r2, epoch)
    log_value('Validation R2', val_r2, epoch)
    log_value('Training Spearman Corr ', train_spearman, epoch)
    log_value('Validation Spearman Corr ', val_spearman, epoch)
    log_value('Training Pearson Corr ', train_pearson, epoch)
    log_value('Validation Pearson Corr ', val_pearson, epoch)
    if val_pearson > best_val:
        best_val = val_pearson
        print('save model: ')
        torch.save(model, './models_hybrid/CQT_CRNN_0611_2')

c:\Users\User\anaconda3\envs\ailab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Epoch:  0
training set: 
Pearson Corr =    0.17511,  P-value = 1.7452895839094653e-12
Spearman Corr =  0.18866,  P-value =  2.7657724628597024e-14
R2 =  -0.07813044370036759
validation set: 
Pearson Corr =    0.44486,  P-value = 4.11918006498314e-11
Spearman Corr =  0.43726,  P-value =  9.548058523736894e-11
R2 =  0.1845923676618555
Train Loss tensor(0.2138, device='cuda:0') 0
Validation Loss tensor(0.1557, device='cuda:0') 0
save model: 


Epoch:  1
training set: 
Pearson Corr =    0.31032,  P-value = 4.678735871390918e-37
Spearman Corr =  0.30196,  P-value =  4.3825448495230366e-35
R2 =  0.07217322800778392
validation set: 
Pearson Corr =    0.51594,  P-value = 5.334138390418772e-15
Spearman Corr =  0.52368,  P-value =  1.7646863647945382e-15
R2 =  0.03543839821411254
Train Loss tensor(0.1840, device='cuda:0') 1
Validation Loss tensor(0.1842, device='cuda:0') 1
save model: 


Epoch:  2
training set: 
Pearson Corr =    0.38138,  P-value = 1.49839867103306e-56
Spearman Corr =  0.3803

KeyboardInterrupt: 

In [1]:
#train  Cqt0611
import torch
import time
import numpy as np
from torch import nn
from torch.autograd import Variable
from models.SpectralCRNN import SpectralCRNN_Reg_Dropout_tsne as SpectralCRNN
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from sklearn import metrics
from torch.optim import lr_scheduler
from scipy.stats import pearsonr as pearsoncorr
from scipy.stats import spearmanr as spearmancorr
import pdb
import random
from dataLoaders.SpectralDataset import SpectralDataset, SpectralDataLoader
from tensorboard_logger import configure, log_value
# Configure tensorboard logger

# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
configure('runs/CQT_CRNN_4', flush_secs=2)  
# Parameteres for Spectral Representation
rep_params = {'method': 'CQT', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}

# Load Datasets
train_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_train_1.dill', 0, rep_params)
train_dataloader = SpectralDataLoader(train_dataset, batch_size=5, num_workers=4, shuffle=True)
#pdb.set_trace()

test_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_test_1.dill', 0, rep_params)
test_dataloader = SpectralDataLoader(test_dataset, batch_size=1, num_workers=1,shuffle=True)

valid_dataset = SpectralDataset('D:/DAMP/Research/dilldata/CRNN_val_1.dill', 0, rep_params)
valid_dataloader = SpectralDataLoader(valid_dataset, batch_size=1, num_workers=1,shuffle=True)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def evaluate_classification(targets, predictions):
    r2 = metrics.r2_score(targets, predictions)
    predictions[predictions < -1] = -1
    predictions[predictions > 1] = 1
    pearson = pearsoncorr(targets.flatten(), predictions.flatten())
    pearson_corre = pearson[0]
    spearman = spearmancorr(targets.flatten(), predictions.flatten())
    print(f'Pearson Corr =  {pearson_corre:9.5f},  P-value = {pearson[1]}')
    print(f"Spearman Corr =  {spearman[0]:7.5f},  P-value =  {spearman[1]}")
    print("R2 = ", r2)
    return r2, pearson_corre,spearman[0]


# Define Model

model = SpectralCRNN().cuda()



criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)

batch_time = AverageMeter()
data_time = AverageMeter()

train_loss = 0
validation_loss = 0

num_epochs = 250
best_val = 0
epoch_time = time.time()
for epoch in range(num_epochs):
    print('\n')
    print('Epoch: ', epoch)
    print('training set: ')
    model.train()
    avg_loss = 0.0
    end = time.time()
    all_predictions = []
    all_targets = []
    losses = AverageMeter()
    for i, (data) in enumerate(train_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end)
        end = time.time()
    train_loss = losses.avg
    train_r2, train_pearson, train_spearman= evaluate_classification(np.array(all_targets), np.array(all_predictions))

    print('validation set: ')
    model.eval()
    losses = AverageMeter()
    all_predictions = []
    all_targets = []
    for i, (data) in enumerate(valid_dataloader):
        inputs, targets = data
        data_time.update(time.time() - end)
        inputs = Variable(inputs.cuda(), requires_grad=False)
        targets = Variable(targets.cuda(), requires_grad=False)
        targets = targets.view(-1, 1)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0]
        all_predictions.extend(out.data.cpu().numpy())
        all_targets.extend(targets.data.cpu().numpy())
        loss = criterion(out, targets)
        loss_value = loss.data
        losses.update(loss_value, inputs.size(0))
    #print('Validating Completed. Loss: {}'.format(losses.avg))
    valid_loss = losses.avg
    val_r2, val_pearson,val_spearman  = evaluate_classification(np.array(all_targets), np.array(all_predictions))
    print('Train Loss', train_loss, epoch)
    print('Validation Loss', valid_loss, epoch)
    log_value('Train Loss', train_loss, epoch)
    log_value('Validation Loss', valid_loss, epoch)
    log_value('Training R2', train_r2, epoch)
    log_value('Validation R2', val_r2, epoch)
    log_value('Training Spearman Corr ', train_spearman, epoch)
    log_value('Validation Spearman Corr ', val_spearman, epoch)
    log_value('Training Pearson Corr ', train_pearson, epoch)
    log_value('Validation Pearson Corr ', val_pearson, epoch)
    if val_pearson > best_val:
        best_val = val_pearson
        print('save model: ')
        torch.save(model, './models_hybrid/CQT_CRNN_0612')

c:\Users\User\anaconda3\envs\ailab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Epoch:  0
training set: 
Pearson Corr =    0.17511,  P-value = 1.7452895839094653e-12
Spearman Corr =  0.18866,  P-value =  2.7657724628597024e-14
R2 =  -0.07813044370036759
validation set: 
Pearson Corr =    0.44486,  P-value = 4.11918006498314e-11
Spearman Corr =  0.43726,  P-value =  9.548058523736894e-11
R2 =  0.1845923676618555
Train Loss tensor(0.2138, device='cuda:0') 0
Validation Loss tensor(0.1557, device='cuda:0') 0
save model: 


Epoch:  1
training set: 
Pearson Corr =    0.31032,  P-value = 4.678735871390918e-37
Spearman Corr =  0.30196,  P-value =  4.3825448495230366e-35
R2 =  0.07217322800778392
validation set: 
Pearson Corr =    0.51594,  P-value = 5.334138390418772e-15
Spearman Corr =  0.52368,  P-value =  1.7646863647945382e-15
R2 =  0.03543839821411254
Train Loss tensor(0.1840, device='cuda:0') 1
Validation Loss tensor(0.1842, device='cuda:0') 1
save model: 


Epoch:  2
training set: 
Pearson Corr =    0.38138,  P-value = 1.49839867103306e-56
Spearman Corr =  0.3803

In [15]:
import scipy.io.wavfile
from dataLoader import SpectralDataset, SpectralDataLoader
from model import SpectralCRNN_Reg_Dropout_tsne as SpectralCRNN
import torch
from torch.autograd import Variable
from torch.autograd import Variable
import torch




def pridiction(wavfile ):
    wavfile=wavfile
    fs, audio = scipy.io.wavfile.read(wavfile) 
    # = audio / 32768.0
    test_tobedumped = []
    test_tobedumped.append({
                        'audio': [audio / 32768.0, fs], 
                        'ratings': [0]
                    })
    params = {'method': 'CQT', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}
    dataset = SpectralDataset(test_tobedumped, 0, params)
    dataloader = SpectralDataLoader(dataset, batch_size=1, num_workers=1,shuffle=True)

    model_path = 'D:/DAMP/Research/final project/model_CQT_CRNN'
    model = SpectralCRNN().cuda()
    model = torch.load(model_path)
    model.eval()


    for i, (data) in enumerate(dataloader):
        inputs, targets = data
        inputs = Variable(inputs.cuda(), requires_grad = False)
        model.init_hidden(inputs.size(0))
        out = model(inputs)[0].tolist()[0][0]

    return out

wavfile  = r"D:\DAMP\SingEval_DAMPcurated\audio_snippets_1\_cups_pitch_perfect\68307032_39376090.wav"
pridiction(wavfile)

ModuleNotFoundError: No module named 'DataLoader'

In [14]:

#rep_params = {'method': 'CQT', 'hop_length': 512, 'n_bins': 96, 'bins_per_octave': 24, 'normalize': True}

    # Load Datasets
#train_dataset = SpectralDataset('D:/DAMP/Research/dilldata/train_1.dill', 0, rep_params)
    
perf_data = dill.load(open('D:/DAMP/Research/dilldata/train_1.dill', 'rb'))
label_id = 0
length = len(perf_data)


perf_data[0]
# idx = 1
# y, sr = perf_data[idx]['audio']
# print('y=',y,', sr=',sr)
# #X = extract_features(y, sr)
# label = perf_data[idx]['ratings'][label_id]
# ph_notes = perf_data[idx]['pitch_histogram'] # 不同
# pitchscore = perf_data[idx]['pitchscore']
# # y = label
# torch.unsqueeze(X, 0), y, ph_notes,pitchscore # 不同

NameError: name 'dill' is not defined

In [5]:
import torchaudio
bundle = torchaudio.pipelines.WAV2VEC2_BASE
model = bundle.get_model()

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960.pth" to C:\Users\User/.cache\torch\hub\checkpoints\wav2vec2_fairseq_base_ls960.pth
100.0%


In [13]:
import torch
import torch.nn as nn


class MosDownstream(nn.Module):
    def __init__(self, upstream_dim, projector_dim, clipping, attention_pooling):
        super(MosDownstream, self).__init__()
        self.connector = nn.Linear(upstream_dim, projector_dim)
        self.model = MosDownstreamModule(
            input_dim = projector_dim,
            clipping = clipping,
            attention_pooling = attention_pooling
        )

    def forward(self, features):
        features = self.connector(features)
        scores = self.model(features)
        return scores

class MosDownstreamModule(nn.Module):
    def __init__(self, input_dim, clipping=False, attention_pooling=False, num_judges=5000, **kwargs):
        super(MosDownstreamModule, self).__init__()
        self.mean_net_linear = nn.Linear(input_dim, 1)
        self.mean_net_clipping = clipping
        self.mean_net_pooling = SelfAttentionPooling(input_dim) if attention_pooling else None
        # self.bias_net_linear = nn.Linear(input_dim, 1)
        # self.bias_net_pooling = SelfAttentionPooling(input_dim) if attention_pooling else None
        # self.judge_embbeding = nn.Embedding(num_embeddings = num_judges, embedding_dim=input_dim)

    def forward(self, features, judge_ids=None):
        if self.mean_net_pooling is not None:
            x = self.mean_net_pooling(features)
            segment_score = self.mean_net_linear(x)
        else:
            x = self.mean_net_linear(features)
            segment_score = x.squeeze(-1).mean(dim=-1)
            
        if self.mean_net_clipping:
            segment_score = torch.tanh(segment_score) * 2 + 3
        
        # if judge_ids is None:
        return segment_score.squeeze(-1)
        
        # else:
        #     time = features.shape[1]
        #     judge_features = self.judge_embbeding(judge_ids)
        #     judge_features = torch.stack([judge_features for i in range(time)], dim = 1)
        #     bias_features = features + judge_features
            
        #     if self.bias_net_pooling is not None:
        #         y = self.bias_net_pooling(bias_features)
        #         bias_score = self.bias_net_linear(y)
        #     else:
        #         y = self.bias_net_linear(bias_features)
        #         bias_score = y.squeeze(-1).mean(dim=-1)
        #     bias_score = bias_score + segment_score

        # return segment_score.squeeze(-1), bias_score.squeeze(-1)

class SelfAttentionPooling(nn.Module):
    """
    Implementation of SelfAttentionPooling
    Original Paper: Self-Attention Encoding and Pooling for Speaker Recognition
    https://arxiv.org/pdf/2008.01077v1.pdf
    """

    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)

    def forward(self, batch_rep):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension

        attention_weight:
            att_w : size (N, T, 1)

        return:
            utter_rep: size (N, H)
        """
        softmax = nn.functional.softmax
        att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)

        return utter_rep

In [12]:
import os
import torchaudio
import IPython
import requests
import torch
import torchaudio

torch.random.manual_seed(123)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bundle = torchaudio.pipelines.WAV2VEC2_BASE
model = bundle.get_model().to(device)

SPEECH_FILE   = "D:/DAMP/SingEval_DAMPcurated/audio_snippets_1/_cups_pitch_perfect/68307032_39376090.wav"
#SPEECH_FILE   = "D:/DAMP/SingEval_DAMPcurated/audio_snippets_1/_let_it_go/70366197_41259492.wav"
bundle = torchaudio.pipelines.WAV2VEC2_BASE
print("Sample Rate:", bundle.sample_rate)
print(model.__class__)

waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
with torch.inference_mode():
    features, _ = model(waveform)
features.shape

Sample Rate: 16000
<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


torch.Size([1, 722, 768])